# Forecasting results

## Model Configuration for Sequence Length 6, Prediction Length 3
| Parameter | Value |
|-----------|-------|
| Learning Rate | 0.0001 |
| Epoch | 20 |
| Optimizer | Adam |
| Loss Function | MSE |
| Sequence Length | 6 |
| Prediction Length | 3 |
| Features | 274 | 

### Results for Sequence Length 6, Prediction Length 3
| Model | MAE | RMSE | sMAPE | rRMSE |
|-------|-----|------|-------|-------|
| LSTM | 1.1138 | 2.9997 | 61.8296 | 29.8876 |
| CHGH | 1.5298 | 4.5268 | 69.2573 | 61.4071 |
| Autoformer | 0.4678 | 1.3097 | 55.7886 | 12.1954 |
| DLinear | 1.9042 | 5.5988 | 73.3004 | 73.7213 |
| Crossformer | 0.9771 | 2.4018 | 61.7157 | 23.5666 |
| FEDformer | 0.2996 | 0.9487 | 54.1525 | 9.0451 |
| FiLM | 1.9947 | 5.5685 | 69.2698 | 57.5120 |
| FreTS | 1.1936 | 2.9220 | 64.3295 | 31.8568 |
| Informer | 0.2189 | 0.6089 | 53.4241 | 6.0289 |
| Informer | 0.2189 | 0.6089 | 53.4241 | 6.0289 |
| Koopa | 1.6917 | 4.9614 | 72.0530 | 60.0705 |
| LightTS | 1.3546 | 3.4511 | 64.1723 | 36.8305 |
| Nonstationary_Transformer | 0.5281 | 1.8353 | 60.3643 | 19.5240 |
| PatchTST | 1.6545 | 4.4558 | 71.4168 | 53.7877 |
| Reformer | 0.2270 | 0.7345 | 53.5509 | 7.3786 |
| SegRNN | 1.6139 | 4.5028 | 67.8708 | 52.0827 |
| TiDE | 2.2119 | 6.4118 | 76.7150 | 81.0349 |
| Transformer | 0.2131 | 0.7003 | 53.3577 | 6.8972 |
| TSMixer | 1.1384 | 3.1691 | 63.5093 | 34.2455 |

---
## Model Configuration for Sequence Length 4, Prediction Length 3
| Parameter | Value |
|-----------|-------|
| Learning Rate | 0.0001 |
| Epoch | 20 |
| Optimizer | Adam |
| Loss Function | MSE |
| Sequence Length | 4 |
| Prediction Length | 3 |
| Features | 274 | 

# Metric Results for Sequence Length 4, Prediction Length 3
| Model | MAE | RMSE | sMAPE | rRMSE |
|-------|-----|------|-------|-------|
| Nonstationary_Transformer | 0.6028 | 1.8229 | 62.6517 | 17.5651 |
| DLinear | 2.4592 | 7.3374 | 83.6295 | 70.7019 |
| CHGH | 1.5295 | 4.5022 | 68.8993 | 43.3829 |
| FEDformer | 0.2854 | 0.7885 | 54.1646 | 7.5977 |
| Reformer | 0.1390 | 0.3786 | 52.6275 | 3.6479 |
| TSMixer | 1.3557 | 4.2139 | 66.6409 | 40.6042 |
| LSTM | 1.0530 | 2.8131 | 61.2632 | 27.1066 |
| FreTS | 1.3040 | 3.3229 | 65.3698 | 32.0190 |
| Transformer | 0.1569 | 0.4905 | 52.8600 | 4.7264 |
| Autoformer | 0.4965 | 1.4145 | 56.6842 | 13.6301 |
| Crossformer | 1.0788 | 2.7027 | 63.3063 | 26.0429 |
| Informer | 0.2099 | 0.5859 | 53.2376 | 5.6458 |
| TiDE | 2.1243 | 6.2068 | 73.1170 | 59.8080 |
| Koopa | 1.8154 | 5.7229 | 72.7798 | 55.1450 |

## Evaluation metrics
To **evaluate** the performance of various benchmark models in job skill demand forecasting tasks, they are two selected metrics:
* [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error): It is calculated over $H$ observations using the formula: 
\begin{equation*}
MAE = \frac{1}{H}\sum_{i=1}^{H}|y_i - \hat{y}_i|
\end{equation*}
* [RMSE](https://en.wikipedia.org/wiki/Root_mean_square_deviation): It is calculated as: 
\begin{equation*}
RMSE = \sqrt{\frac{1}{H}\sum_{i=1}^{H}(y_i - \hat{y}_i)^2}
\end{equation*} 
Both `MAE` and `RMSE` are scale-dependent metrics, which makes them unsuitable for comparison across different granularities. Additionally, these metrics are less sensitive to prediction errors at lower skill demand values. 

Therefore, they also applied:
* [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error): Which considers both the magnitude and direction of errors, making it suitable for comparing forecasts across different scales.
\begin{equation*}
SMAPE = \frac{2}{H}*\sum_{i = 1}^{H}\frac{|y_i - \hat{y}_i|}{|y_i| + |\hat{y}_i|}
\end{equation*}
* `RRMSE`: Measures the square root of the average of the squared percentage errors.
\begin{equation*}
RRMSE = \sqrt{\frac{\frac{1}{H}\sum_{i = 1}^{H}(y_i - \hat{y}_i)^2}{\sum_{i = 1}^{H}(\hat{y}_i)^2}}
\end{equation*}

---

In [91]:
from metrics import mae, rmse, smape, rrmse, compute_metrics
import numpy as np
import re

FILE_WRITING = "../doc/metric_results.md"
rgx = "(?<=forecast_0)(.*?)(?=job_demand)"

In [92]:
def get_metrics(path : str) -> dict:
    pred, true = np.load(path + '/pred.npy'), np.load(path + '/true.npy')

    if FILE_WRITING:
        with open(FILE_WRITING, 'a') as f:
            model_name = path.split('/')[-1]
            metrics = compute_metrics(pred, true)
            model_name = re.search(rgx, model_name).group(0)[1:-1]
            f.write(f"| {model_name} | {metrics['mae']:.4f} | {metrics['rmse']:.4f} | {metrics['smape']:.4f} | {metrics['rrmse']:.4f} |\n")

    return compute_metrics(pred, true)

## LSTM

In [114]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_LSTM_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.113809),
 'rmse': np.float32(2.9996738),
 'smape': np.float32(61.829643),
 'rrmse': np.float32(29.887575)}

## Cross-View Hierarchichal Graph Learning Hypernetwork

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_CHGH_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.5297962),
 'rmse': np.float32(4.526833),
 'smape': np.float32(69.25732),
 'rrmse': np.float32(61.407078)}

## Autoformer

In [115]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_Autoformer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(0.4677701),
 'rmse': np.float32(1.3097129),
 'smape': np.float32(55.78857),
 'rrmse': np.float32(12.195366)}

## DLinear

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_DLinear_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.9041979),
 'rmse': np.float32(5.598809),
 'smape': np.float32(73.30041),
 'rrmse': np.float32(73.72125)}

## Crossformer

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_Crossformer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")


{'mae': np.float32(0.97710013),
 'rmse': np.float32(2.40176),
 'smape': np.float32(61.715706),
 'rrmse': np.float32(23.566608)}

## FEDformer

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_FEDformer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(0.29957002),
 'rmse': np.float32(0.94869506),
 'smape': np.float32(54.15249),
 'rrmse': np.float32(9.045085)}

## FiLM

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_FiLM_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.9946569),
 'rmse': np.float32(5.568467),
 'smape': np.float32(69.26977),
 'rrmse': np.float32(57.51198)}

## FreTS

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_FreTS_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.1936169),
 'rmse': np.float32(2.9220037),
 'smape': np.float32(64.32952),
 'rrmse': np.float32(31.856792)}

## Informer

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_Informer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(0.2188686),
 'rmse': np.float32(0.60892594),
 'smape': np.float32(53.424107),
 'rrmse': np.float32(6.0288525)}

## iTransformer

In [116]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_Informer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(0.2188686),
 'rmse': np.float32(0.60892594),
 'smape': np.float32(53.424107),
 'rrmse': np.float32(6.0288525)}

## Koopa

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_Koopa_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")


{'mae': np.float32(1.6916887),
 'rmse': np.float32(4.9614115),
 'smape': np.float32(72.053024),
 'rrmse': np.float32(60.070473)}

## LightTS

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_LightTS_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.3545932),
 'rmse': np.float32(3.4511151),
 'smape': np.float32(64.17227),
 'rrmse': np.float32(36.830482)}

## Nonstationary_Transformer

In [ ]:
get_metrics("../models/multivariate_time_series/results/lseq_6_len_3/ong_term_forecast_0_Nonstationary_Transformer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(0.52811134),
 'rmse': np.float32(1.8352921),
 'smape': np.float32(60.36432),
 'rrmse': np.float32(19.524033)}

## PatchTST

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_PatchTST_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.6544572),
 'rmse': np.float32(4.4557977),
 'smape': np.float32(71.41676),
 'rrmse': np.float32(53.787655)}

## Reformer

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_Reformer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(0.22699843),
 'rmse': np.float32(0.7345094),
 'smape': np.float32(53.55089),
 'rrmse': np.float32(7.37856)}

## SegRNN

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_SegRNN_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.6138557),
 'rmse': np.float32(4.5027585),
 'smape': np.float32(67.8708),
 'rrmse': np.float32(52.08275)}

## TiDE

In [117]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_TiDE_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(2.211866),
 'rmse': np.float32(6.4118013),
 'smape': np.float32(76.71502),
 'rrmse': np.float32(81.03487)}

## Transformer

In [ ]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_Transformer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(0.21306671),
 'rmse': np.float32(0.7003056),
 'smape': np.float32(53.357666),
 'rrmse': np.float32(6.897191)}

## TSMixer

In [118]:
get_metrics("../models/multivariate_time_series/results/seq_6_len_3/long_term_forecast_0_TSMixer_job_demand_region_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_0")

{'mae': np.float32(1.1383607),
 'rmse': np.float32(3.1690867),
 'smape': np.float32(63.509308),
 'rrmse': np.float32(34.245457)}

## Models that doesn't work
The following models were attemped but didn't work properly, ironically they are also not in the Job-SDF paper's benchmark:
* MICN
* ETSformer
* Pyraformer
* TemporalFusionTransformer
* TimesNet